In [48]:
import pandas as pd
import numpy as np
import zipfile
import os,re

from termcolor import colored

In [61]:
DICT_WIKI_PATH = 'data/dictionary_lowercase.zip'
DICT_WORD_PATH = 'data/wikipedia_en.zip'
DICT_MAIN_PATH = 'data/dict_main.zip'

In [50]:
dict_wiki = zipfile.ZipFile(DICT_WIKI_PATH)
dict_wiki = pd.read_csv(dict_wiki.open(re.sub('zip$','txt',DICT_WIKI_PATH)),header=None,names=['word'])
dict_wiki['word'] = dict_wiki['word'].apply(lambda x: str(x).upper())
dict_wiki = dict_wiki.drop_duplicates()

dict_word = zipfile.ZipFile(DICT_WORD_PATH)
dict_word = pd.read_csv(dict_word.open(re.sub('zip$','txt',DICT_WORD_PATH)),sep=' ',header=None,names=['word','len','instances','articles'])
dict_word['word'] = dict_word['word'].apply(lambda x: str(x).upper())
dict_word = dict_word.groupby(['word','len']).sum().sort_values(by='instances',ascending=False).reset_index()

In [51]:
dict_main = dict_word.merge(dict_wiki,how='inner',on='word',sort=False)

In [62]:
dict_main.to_csv(DICT_MAIN_PATH,compression=dict(method='zip',archive_name=re.sub('zip$','csv',DICT_MAIN_PATH)))

In [53]:
dict_main

,word,len,instances,articles
0,THE,3,32823802,4968822
1,OF,2,17445728,4279162
2,IN,2,15303927,4539682
3,AND,3,13537194,3886207
4,IS,2,8334346,4234968
...,...,...,...,...
54847,SCARIFY,7,2,2
54848,SCARILY,7,2,2
54849,SCANTINESS,10,2,2
54850,HENPECK,7,2,2


In [54]:
class Wordle:
    def __init__(self,data,n=5):
        self.data = data[data['len']==n].reset_index(drop=True)
        self.word = self.data['word'][np.random.randint(len(self.data))]

    def guess(self):
        t = 1
        keys = {}
        colors = ['\033[38;5;241m','\033[38;5;254m','\033[38;5;214m','\033[38;5;36m']
        for a in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
            keys[a] = 0
        match = np.zeros(len(self.word),dtype=int)
        while min(match) < 3:
            m = ''
            myguess = input('Guess '+str(t)+':')
            if len(myguess) != len(self.word):
                print('Please choose a '+str(len(self.word))+'-letter word!')
                continue
            else:
                for c in range(len(myguess)):
                    if myguess[c] == self.word[c]:
                        match[c] = keys[myguess[c]] = 3
                    elif (myguess[c] in self.word) and (myguess[c] != self.word[c]):
                        match[c] = keys[myguess[c]] = 2
                    else:
                        match[c] = keys[myguess[c]] = 1
                if min(match) != 3:
                    t += 1
                for s in myguess:
                    m += '\033[1m'+colors[keys[s]]+' '+s
                m += '   :   '
                for s in keys.keys():
                    m += colors[keys[s]]+' '+s
                print(m)
        return 'you got it in '+str(t)+' tries :)'

In [55]:
word = Wordle(dict_main)

In [57]:
word.guess()

In [59]:
if __name__ == '__main__':
    dict_main = pd.read_csv(zipfile.ZipFile(DICT_MAIN_PATH).open(re.sub('zip$','csv',DICT_MAIN_PATH)))
    word = Wordle(dict_main)
    word.guess()